# Automated License Plate Detection using LPRNet
LPRNet is a lightweight convolutional neural network designed for
end-to-end license plate recognition. The model formulates recognition
as a sequence prediction problem and employs CTC loss to avoid explicit
character segmentation.

Given a cropped license plate image, LPRNet directly outputs the
corresponding alphanumeric sequence.

**Execution Environment:** Google Colab (GPU-enabled)


In [ ]:
class LicensePlateValidator:
    """
    Validates license plate formats and detects invalid/redundant entries
    """

    def __init__(self):
        # Common license plate patterns (can be customized)
        # Format: Country_Code: Pattern
        self.patterns = {
            'INDIA': r'^[A-Z]{2}\d{2}[A-Z]{2}\d{4}$',  # KA01AB1234
            'US': r'^[A-Z0-9]{1,8}$',  # Generic US format
            'UK': r'^[A-Z]{2}\d{2}[A-Z]{3}$',  # UK format
            'GENERIC': r'^[A-Z0-9]{3,10}$'  # Generic alphanumeric
        }

        self.detected_plates = {}
        self.duplicates = []
        self.invalid_entries = []
        self.valid_entries = []

    def is_valid_format(self, plate_number, format_type='GENERIC'):
        """
        Check if plate number matches valid format
        """
        try:
            pattern = self.patterns.get(format_type, self.patterns['GENERIC'])
            cleaned = plate_number.strip().upper()

            # Remove common OCR errors
            cleaned = self._correct_ocr_errors(cleaned)

            is_valid = bool(re.match(pattern, cleaned))
            return cleaned, is_valid
        except Exception as e:
            print(f"Error validating plate: {e}")
            return plate_number, False

    def _correct_ocr_errors(self, text):
        """
        Correct common OCR misreadings
        """
        corrections = {
            'O': '0',  # O to 0
            'I': '1',  # I to 1
            'S': '5',  # S to 5
            'Z': '2',  # Z to 2
            'B': '8',  # B to 8
        }

        result = text
        # This is a simplified correction; refine based on your use case
        return result

    def check_duplicate(self, plate_number):
        """
        Check if plate number already exists in database
        """
        cleaned_plate = plate_number.strip().upper()
        if cleaned_plate in self.detected_plates:
            return True
        return False

    def add_plate(self, plate_number, image_name, confidence, format_type='GENERIC'):
        """
        Add plate to database with validation
        """
        cleaned_plate, is_valid = self.is_valid_format(plate_number, format_type)

        entry = {
            'plate_number': cleaned_plate,
            'image_name': image_name,
            'confidence': confidence,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'is_valid': is_valid
        }

        # Check for duplicates
        if self.check_duplicate(cleaned_plate):
            self.duplicates.append(entry)
            return 'duplicate'

        # Check for invalid format
        if not is_valid:
            self.invalid_entries.append(entry)
            return 'invalid'

        # Valid and unique entry
        self.detected_plates[cleaned_plate] = entry
        self.valid_entries.append(entry)
        return 'valid'

class LPRNet(nn.Module):
    def __init__(self, num_classes=len(CHARS)):
        super().__init__()

        self.backbone = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d((1, 3), (1, 3)),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d((2, 3), (2, 3)),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.classifier = nn.Conv2d(256, num_classes, kernel_size=(1, 1))

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)              # (B, C, H, W)
        x = x.mean(dim=2)                   # collapse height
        x = x.permute(2, 0, 1)              # (W, B, C)
        return x
    def ctc_decode(preds):
    preds = preds.argmax(dim=2).squeeze(1).cpu().numpy()

    decoded = []
    prev = -1
    for p in preds:
        if p != prev and p != 0:
            decoded.append(CHARS[p])
        prev = p

    return "".join(decoded)

def recognize_plates(image_path, model, device="cpu"):
    image = cv2.imread(image_path)
    plates = detect_plates(image)

    results = []

    for (x1, y1, x2, y2) in plates:
        plate = image[y1:y2, x1:x2]

        if plate.size == 0:
            continue

        input_tensor = preprocess_plate(plate).to(device)

        with torch.no_grad():
            preds = model(input_tensor)

        plate_text = ctc_decode(preds)
        results.append(plate_text)

    return results

